### Import Modules

In [ ]:
from nltk.tokenize import word_tokenize


import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import numpy as array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer


### Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset= pd.read_csv('/content/drive/MyDrive/4-1_Semester/practice code/Sentiment Analysis/IMDB Dataset.csv')
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
dataset.sentiment = dataset.sentiment.apply(lambda x: 0 if x =='negative' else 1)
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
dataset.shape
dataset.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

### Processing Data

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
corpus=[]

for i in range(0, 50000):
  review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i])
  review = re.sub(r"\s+[a-zA-Z]\s", ' ', review)
  review = re. sub(r'\s+', ' ',review)
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)
  print(len(corpus), end=' -- ') 
  print(len(corpus[i]))

Streaming output truncated to the last 5000 lines.
45001 -- 431
45002 -- 732
45003 -- 202
45004 -- 466
45005 -- 481
45006 -- 966
45007 -- 533
45008 -- 1523
45009 -- 328
45010 -- 424
45011 -- 695
45012 -- 529
45013 -- 144
45014 -- 432
45015 -- 539
45016 -- 449
45017 -- 907
45018 -- 2221
45019 -- 223
45020 -- 1493
45021 -- 267
45022 -- 164
45023 -- 403
45024 -- 729
45025 -- 702
45026 -- 1249
45027 -- 594
45028 -- 1031
45029 -- 2455
45030 -- 370
45031 -- 107
45032 -- 3119
45033 -- 477
45034 -- 143
45035 -- 1153
45036 -- 430
45037 -- 1026
45038 -- 420
45039 -- 469
45040 -- 757
45041 -- 347
45042 -- 801
45043 -- 584
45044 -- 451
45045 -- 434
45046 -- 1169
45047 -- 2735
45048 -- 286
45049 -- 588
45050 -- 800
45051 -- 745
45052 -- 472
45053 -- 1781
45054 -- 251
45055 -- 310
45056 -- 398
45057 -- 778
45058 -- 228
45059 -- 424
45060 -- 357
45061 -- 243
45062 -- 685
45063 -- 641
45064 -- 476
45065 -- 436
45066 -- 901
45067 -- 1292
45068 -- 386
45069 -- 204
45070 -- 518
45071 -- 350
45072 -- 210


In [ ]:
corpus[0]

'one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust not show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci not high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc not violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that 

### Counter Vector Embadding

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [ ]:
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values
y = y.astype('int')

In [ ]:
print(X)

print("===============")

print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 1 1 ... 0 0 0]


In [ ]:
print(len(X))

print("===============")

print(len(y))

50000
50000


### Train - Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.20, random_state= 42)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(40000, 1420) (40000,)
(10000, 1420) (10000,)


In [ ]:
X_train.shape

(40000, 1420)

### Model Fit - CNN

In [ ]:
from keras.layers import Input
model_CNN = Sequential()
model_CNN. add (Embedding(input_dim = len(corpus)+1, output_dim = 50, input_length = 1420, trainable=False))

model_CNN. add (Conv1D(filters=128, kernel_size= 5,padding ='same', activation= 'relu'))
model_CNN. add (Dense(24, activation= 'relu'))
model_CNN. add (GlobalMaxPool1D())
model_CNN. add (Dropout(0.3))
model_CNN. add (Dense(1, activation= 'sigmoid'))


# model_CNN. add (Conv1D(filters=64, kernel_size= 5,padding ='same', activation= 'relu'))
# model_CNN. add (MaxPooling1D(2))
# model_CNN. add(Dropout(0.5))

# model_CNN. add (Dense(32, activation= 'relu'))
# model_CNN. add(Dropout(0.5))
# model_CNN. add (Dense(16, activation= 'relu'))
# model_CNN. add (GlobalMaxPool1D())
# model_CNN. add (Dense(4, activation= 'softmax'))

# model_CNN. add (Conv1D(filters=32, kernel_size=3, padding = 'same',activation = 'relu'))
# model_CNN. add (MaxPooling1D(pool_size = 2))
# model_CNN. add (Conv1D(filters=32, kernel_size=3, padding = 'same',activation = 'relu'))
# model_CNN. add (Dense(1, activation= 'sigmoid'))

model_CNN.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [ ]:
#model_CNN.build(X_train.shape)
model_CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1420, 50)          2500050   
                                                                 
 conv1d (Conv1D)             (None, 1420, 128)         32128     
                                                                 
 dense (Dense)               (None, 1420, 24)          3096      
                                                                 
 global_max_pooling1d (Globa  (None, 24)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                        

In [ ]:
history = model_CNN.fit(X_train, y_train, epochs=10, batch_size=4, verbose=1, validation_split= 0.2)
score =model_CNN.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
8000/8000 [==============================] - 264s 33ms/step - loss: 0.6936 - accuracy: 0.5014 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
8000/8000 [==============================] - 257s 32ms/step - loss: 0.6933 - accuracy: 0.4958 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/10
8000/8000 [==============================] - 250s 31ms/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
8000/8000 [==============================] - 256s 32ms/step - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
8000/8000 [==============================] - 252s 31ms/step - loss: 0.6934 - accuracy: 0.4978 - val_loss: 0.6930 - val_accuracy: 0.5082
Epoch 6/10
8000/8000 [==============================] - 253s 32ms/step - loss: 0.6933 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.5038
Epoch 7/10
8000/8000 [==============================] - 250s 31ms/step - loss: 0.6932 - accuracy: 0.5057 - val_loss: 0

### Data Validation

In [ ]:
print("Test Score: ",score[0])
print("Test Accuracy: ", score[1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')

plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title("Model Loss")
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend(['train', 'test'], loc = 'upper left')

plt.show()

### Other Scores

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, f1_score, precision_score, recall_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_recall_fscore_support

def evaluate(y_test, y_pred):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """

    print(cm)
    precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average = 'binary')
    print(f"precision:{precision:.3f} \nrecall:{recall:.3f}\nF1 score: {f1:.3f}")

    # preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')

    
    # Get accuracy over the test set
    # y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('CNN')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
evaluate(y_test, y_pred)